# PSAC 서버에 데이터 옮기기
- 내가 갖고 있는 1000개의 Validation Data를 파싱해서 PSAC 서버에 업로드해야한다.
- C# WPF에서 PSAC 서버에 요청을 해서 데이터를 불러오게 할 것이다.

In [1]:
import joblib

data = joblib.load('Validation_Data.pkl')
data.head()

,Class,Caption,ImageWidth,ImageHeight,Age,ImageFile,PatientName
0,1,This plain abdominal supine radiograph shows m...,512,512,5,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,이태민
1,1,This supine plain abdominal radiograph shows m...,512,512,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,강호수
2,1,This plain abdominal supine radiograph of an a...,512,512,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,강호민
3,1,This plain supine abdominal radiograph of an a...,512,512,7,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,최경태
4,1,This plain supine abdominal radiograph shows m...,512,512,3,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,임도선


# DataFrame to DICOM

In [3]:
import os
import uuid
import datetime
import numpy as np
from PIL import Image
import pydicom
from pydicom.dataset import Dataset, FileDataset

def png_to_color_dicom(row, save_path):
    img = Image.open(row['ImageFile']).convert('RGB')
    pixel_array = np.array(img)

    file_meta = pydicom.Dataset()
    file_meta.MediaStorageSOPClassUID = pydicom.uid.SecondaryCaptureImageStorage
    file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    file_meta.ImplementationClassUID = pydicom.uid.PYDICOM_IMPLEMENTATION_UID

    ds = FileDataset(save_path, {}, file_meta=file_meta, preamble=b"\0" * 128)

    # 필수 태그 추가
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
    ds.SOPClassUID = file_meta.MediaStorageSOPClassUID

    ds.Modality = 'OT'
    ds.Rows, ds.Columns = pixel_array.shape[:2]
    ds.PatientAge = f"{int(row['Age']):03}Y"
    ds.StudyDate = datetime.date.today().strftime('%Y%m%d')

    ds.SpecificCharacterSet = 'ISO_IR 192'  # UTF-8 한글 지원
    ds.PatientName = str(row.get('PatientName', f"Patient_{uuid.uuid4()}"))
    ds.PatientID = str(row.get('PatientID', f"ID_{uuid.uuid4()}"))

    ds.SamplesPerPixel = 3
    ds.PhotometricInterpretation = "RGB"
    ds.PlanarConfiguration = 0
    ds.BitsAllocated = 8
    ds.BitsStored = 8
    ds.HighBit = 7
    ds.PixelRepresentation = 0

    ds.PixelData = pixel_array.tobytes()
    ds.save_as(save_path)



def convert_dataframe_to_dicom(df, output_dir):
    """
    DataFrame 전체를 순회하며
    각 행을 DICOM 파일로 저장하는 함수
    """
    os.makedirs(output_dir, exist_ok=True)

    for idx, row in df.iterrows():
        dicom_path = os.path.join(output_dir, f"image_{idx:04d}.dcm")
        png_to_color_dicom(row, dicom_path)


# 예시로 df가 이미 로드되어 있다고 가정
# df = pd.read_csv('your_csv_or_other_source.csv')

# 출력 폴더 설정
output_folder = './dicom_output'

# 변환 실행
convert_dataframe_to_dicom(data, output_folder)

In [32]:
ds = pydicom.dcmread("dicom_output\image_0001.dcm")
print(ds)  # DICOM 메타데이터 전체 출력

Dataset.file_meta -------------------------------
(0002,0002) Media Storage SOP Class UID         UI: Secondary Capture Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.73714185418037749128523977475146727289
(0002,0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.8.498.1
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 192'
(0008,0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008,0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.8.498.73714185418037749128523977475146727289
(0008,0020) Study Date                          DA: '20250708'
(0008,0060) Modality                            CS: 'OT'
(0008,103E) Series Description                  LO: 'This supine plain abdominal radiograph shows markedly distended stomach.'
(0010,0010) Patient's Name                      PN: '강호수'
(0010,0020) Patient ID                          

# PSAC 서버에 업로딩 성공 (Validation Image data 1000개)

In [ ]:
# import requests
# import json

# # Orthanc 서버 정보
# ORTHANC_URL = "http://localhost:8042"  # 또는 http://your.server.ip:8042
# AUTH = ("orthanc", "orthanc")  # 기본값

# # 1. 환자 목록 가져오기
# patients_url = f"{ORTHANC_URL}/patients"
# response = requests.get(patients_url, auth=AUTH)

# patient_ids = response.json()
# print("환자 수:", len(patient_ids))

# # 2. 각 환자별 정보 조회
# for patient_id in patient_ids:
#     patient_info_url = f"{ORTHANC_URL}/patients/{patient_id}"
#     patient_info = requests.get(patient_info_url, auth=AUTH).json()

#     print(f"\n[환자 이름] {patient_info.get('MainDicomTags', {}).get('PatientName')}")
#     print(f"Study 수: {len(patient_info.get('Studies', []))}")

#     # 3. Study → Series → Instance 순으로 접근
#     for study_id in patient_info['Studies']:
#         study_url = f"{ORTHANC_URL}/studies/{study_id}"
#         study_info = requests.get(study_url, auth=AUTH).json()

#         for series_id in study_info['Series']:
#             series_url = f"{ORTHANC_URL}/series/{series_id}"
#             series_info = requests.get(series_url, auth=AUTH).json()

#             for instance_id in series_info['Instances']:
#                 # 이미지 썸네일 가져오기 (예시)
#                 preview_url = f"{ORTHANC_URL}/instances/{instance_id}/preview"
#                 image = requests.get(preview_url, auth=AUTH).content

#                 # 저장해보기
#                 with open(f"DICOM_GET/{instance_id}.jpg", "wb") as f:
#                     f.write(image)
#                 print(f"  ➤ 이미지 저장됨: /{instance_id}.jpg")


환자 수: 1000

[환자 이름] 이하민
Study 수: 1
  ➤ 이미지 저장됨: /07d4e91d-364f4617-388a192f-e84f04a8-5e55c34f.jpg

[환자 이름] 장수선
Study 수: 1
  ➤ 이미지 저장됨: /4c7d4a66-03e10a0c-3d4368cd-ec888674-0b1a4c13.jpg

[환자 이름] 윤호민
Study 수: 1
  ➤ 이미지 저장됨: /64a69ef2-e55dd7c4-99499399-fcd7bcfd-0010531d.jpg

[환자 이름] 조호선
Study 수: 1
  ➤ 이미지 저장됨: /40b3f3c2-2d562a50-7a7f9cfe-4323ea3b-1af8bec8.jpg

[환자 이름] 강선영
Study 수: 1
  ➤ 이미지 저장됨: /b5535e10-ff665376-9b053bd3-35e3d61a-4b9ab73c.jpg

[환자 이름] 임태성
Study 수: 1
  ➤ 이미지 저장됨: /19a194e4-2be9809a-ab01dc6f-32c13432-f2b44668.jpg

[환자 이름] 이윤호
Study 수: 1
  ➤ 이미지 저장됨: /55026d63-a02f3d5d-c0fdc6fc-da547bcf-8aafc947.jpg

[환자 이름] 박현성
Study 수: 1
  ➤ 이미지 저장됨: /2a5db231-eb0cef9c-246d431a-cb0f443e-9c05bc97.jpg

[환자 이름] 임하호
Study 수: 1
  ➤ 이미지 저장됨: /a177a3b5-ae90b57e-73ae84e7-6f0f3a36-1ada84eb.jpg

[환자 이름] 박민윤
Study 수: 1
  ➤ 이미지 저장됨: /e8d3ad03-99fa048b-7499ee63-f3533dad-fc55b00a.jpg

[환자 이름] 이태우
Study 수: 1
  ➤ 이미지 저장됨: /b00b9c95-f79a713a-b7854b9a-ec21b090-d671fc41.jpg

[환자 이름] 김성수
Study 수: 1
  ➤ 이미지 

KeyboardInterrupt: 